## 인공신경망을 이용한 유방암 분류 (텐서 플로 1.x) 
- 134 ~ 156

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
from sklearn import datasets
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import pandas as pd

In [ ]:
tf.enable_eager_execution()

In [ ]:
data = datasets.load_breast_cancer()

In [ ]:
X = data.data
y = data.target

X = np.array(X, dtype = 'float32')
y = np.array(y, dtype = "float32")

In [ ]:
print("기존 y.shape : ",y.shape)

y = y.reshape(-1,1)

print("변경 y.shape : ",y.shape)

기존 y.shape :  (569,)
변경 y.shape :  (569, 1)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
scaler = StandardScaler()

scaler.fit(X_train)

X_traIn = scaler.transform(X_train)


In [ ]:
def sigmoid(h):
  return 1 /(1 + np.exp(-h))

In [ ]:
initializer =  tf.contrib.layers.xavier_initializer()

w = tf.Variable(initializer([30, 1]))

b =  tf.Variable(initializer([1]))

In [ ]:
optimizer = tf.train.AdamOptimizer(0.001)


In [ ]:
for step in range(100):
  print("="*100)
  print("step = ", step)
  with tf.GradientTape() as tape:
    hypothesis = tf.sigmoid(tf.matmul(X_train, w) + b)
    
    #print("hypothesis(예측) : ", hypothesis)
    cost = -tf.reduce_mean(y_train * tf.log(hypothesis)+(1 - y_train)* tf.log(1 - hypothesis))
    print("cost (MSE) : " , cost)

    grads = tape.gradient(cost, [w,b])
    #print("grads (w, b 기울기) : ", grads)

  optimizer.apply_gradients(grads_and_vars=zip(grads,[w,b]))
  print("update된 w :", w)
  print("update된 b :", b)

In [ ]:
X_test = scaler.transform(X_test)

predict = tf.sigmoid(tf.matmul(X_test,w) + b)

predict01 = tf.cast(predict > 0.5, dtype = tf.float32)

print(predict01)

ac01 = tf.equal(predict01, y_test)

print(ac01)

ac02 = tf.cast(ac01, dtype ='float32')

ac03 = tf.reduce_mean(ac02)

In [ ]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict01, y_test), dtype = tf.float32))

print(accuracy)

tf.Tensor(0.36363637, shape=(), dtype=float32)


## 인공신경망을 이용한 유방암 분류 (텐서 플로 2.x)

- 158~172


In [ ]:
import tensorflow as tf
from sklearn import datasets
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import pandas as pd

In [ ]:
tf.__version__

'2.6.0'

In [ ]:
# 데이터 로드
data = datasets.load_breast_cancer()

# 독립
X = data.data

# 종속
y = data.target

# 스플릿
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
# Standard Scler
# (데이터 -  평균)/표준편차(std)
# 평균 0 , 분산 1로 조정

scaler = StandardScaler()

# 계산
scaler.fit(X_train)

# X_train 변경
X_traIn = scaler.transform(X_train)


In [ ]:
# 순차모델 객체 생성
model = Sequential()

# input_dim = 30 --> 입력데이터 X , 칸의 수는 30
# Dense(1)  = 출력데이터는 y, 칸이 수는 1
model.add(Dense(1, input_dim = 30, activation = 'sigmoid'))

model.summary()

# 경사하강법 설정
model.compile(loss = 'binary_crossentropy', optimizer = Adam(learning_rate = 0.001),metrics = ['acc'])

# 학습
model.fit(X_train, y_train, epochs = 1000)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1)                 31        
Total params: 31
Trainable params: 31
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
# 예측할 X_test 범위 변경
X_test = scaler.transform(X_test)

In [ ]:
pred = model.predict(X_test)

# 예측값이 0.5 이상이면 1, 아니면 0
predict01 = np.where(pred > 0.5,1,0)

print("predict01")
print(predict01)

# 1차원 배열로 변환
predict02 = predict01.flatten()
print(predict02)

# 맞으면 True, 아니면 False
predict03 = (predict02 == y_test)
predict03

# True를 sum하면 True = 1, False = 0
# 
acc = np.sum(predict03) / len(predict03)
print("정확도 : ",acc)